In [1]:
import os
import glob
import torch
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
from torchnet import meter
from progressbar import * #进度条
from natsort import natsorted
import torch.nn.functional as F #torch是关于运算的包
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集

#如果多gpu运行，屏蔽下一句
#os.environ['CUDA_VISIBLE_DEVICES']='2' 

In [2]:
class wide_basic(nn.Module):
    def __init__(self, in_planes, out_planes, dropout_rate, stride=1):
        super(wide_basic, self).__init__()
        self.left = nn.Sequential(
            nn.BatchNorm2d(in_planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=2, bias=True),
            nn.Dropout(p=dropout_rate),
            nn.BatchNorm2d(out_planes),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=True)
        )
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=True)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=True))
    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        #out = self.left(x)
        out += self.shortcut(x)
        
        return out

In [3]:
class Wide_ResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes=10):
        super(Wide_ResNet, self).__init__()
        self.in_planes = 16
        assert ((depth-2)%6 == 0), 'Wide-resnet depth should be 6n+2'
        n = int((depth-2)/6)
        k = widen_factor
        print('| Wide-Resnet %dx%d' % (n, k))
        n_stages = [16, 16*k, 32*k, 64*k]
        self.conv1 = nn.Conv2d(3, n_stages[0], kernel_size=3, stride=1, padding=1, bias=True)
        self.layer1 = self._wide_layer(wide_basic, n_stages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(wide_basic, n_stages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(wide_basic, n_stages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(n_stages[3], momentum=0.9)
        self.linear = nn.Linear(n_stages[3], num_classes)
    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        #print(num_blocks)
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        
        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out

In [4]:
#相比于ResNet只是在 原有基础上加深了卷积层数
#print(Wide_ResNet(26, 10, 0.3, 10))

In [5]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [6]:
class TrainAndValData(Dataset):
    def __init__(self, img_path, csv_path, train=True, transforms=None):
        '''
        获得所有图片路径，并划分训练集、验证集
        '''
        self.train = train
        files = natsorted(glob.glob(img_path + '/*'))
        labels = pd.read_csv(csv_path).values[:, 1]
        files_num = len(files)
        break_point = int(0.9*files_num)
        if self.train:
            self.img_name = files[: break_point]
            self.img_label = labels[: break_point]
        else:
            self.img_name = files[break_point: ]
            self.img_label = labels[break_point: ]
         
   

        #数据增强
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            #训练集用数据增强
            if self.train:
                self.transforms = T.Compose([
                    #T.RandomCrop(224, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32`，
                    T.Resize(40),
                    T.RandomResizedCrop(32),
                    T.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
                    #T.RandomVerticalFlip(),
                    T.ToTensor(),
                    normalize 
                ])
            else:
                self.transforms = T.Compose([
                    T.Resize(32),
                    #T.CenterCrop(224),#中心裁剪
                    T.ToTensor(),
                    normalize 
                ])
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.img_name)
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        '''
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        label = label2int[self.img_label[index]]
        return img, label
class TestData(Dataset):
    def __init__(self, img_path, transforms=None):
        files = natsorted(glob.glob(img_path + '/*'))
        self.img_name = files
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            self.transforms =  T.Compose([
                T.Resize(32),
                T.ToTensor(),
                normalize
            ])
    def __len__(self):
        return len(self.img_name)
    def __getitem__(self, index):
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

In [7]:
train_img_path = 'data/train'
csv_path = 'data/trainLabels.csv'
test_img_path = 'data/test'
train_dataset = TrainAndValData(train_img_path, csv_path, train=True)
val_dataset = TrainAndValData(train_img_path, csv_path, train=False)
test_dataset = TestData(test_img_path)

In [8]:
print('train len is %d' % len(train_dataset))
print(train_dataset[0][0].shape)
print(train_dataset[0][1])#打印标签
print('val len is %d' % len(val_dataset))
print(val_dataset[0][0].shape)
print(val_dataset[0][1])
print('test len is %d' % len(test_dataset))
print(test_dataset[0].shape)

train len is 45000
torch.Size([3, 32, 32])
6
val len is 5000
torch.Size([3, 32, 32])
7
test len is 300000
torch.Size([3, 32, 32])


In [11]:
# 超参数设置
EPOCH = 135   #遍历数据集次数
BATCH_SIZE = 120      #批处理尺寸(batch_size)
#LR = 0.001        #学习率
lr = 0.001
lr_decay = 0.995
weight_decay = 1e-4
model_path = 'model/wide_resnet/wide_resnet.pkl'

In [12]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8) 

device = torch.device("cuda:2")
#net = ResNet_18_34(34)

net = Wide_ResNet(26, 10, 0.3, 10)
#net = net.to(device)
##单GPU
#net = net.cuda()
##多GPU
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net, device_ids=[0,1])
net = net.cuda()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
#optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
#保存模型判断条件
max_val_acc = 0
pre_epoch = 0
max_interval_epoch = 20
pre_train_loss = 100000

print("Start Training...")
for epoch in range(200):
    #训练集
    train_loss = 0
    train_count = 0
    net.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #更新指标
        train_count += labels.size(0)
        train_loss += loss.item()
    train_loss /= train_count

    #验证集
    val_acc = 0
    val_loss = 0
    val_count = 0
    net.eval()
    for i, data in enumerate(valloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels =  inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #更新指标
        val_count += 1
        val_loss += loss.item()
        _, predict = outputs.max(1)
        val_count += labels.size(0)
        val_acc += (predict == labels).sum().item()
    val_acc /= val_count
    val_loss /= val_count
    # print the loss and accuracy
    print('the epoch %d, the train loss is %f, the test loss is %f, the test acc is %f' % (epoch, train_loss, val_loss, val_acc))

    #保存模型
    if val_acc > max_val_acc:
        max_val_acc = val_acc
        pre_epoch = epoch
        torch.save(net, model_path)#保存整个神经网络的的结构信息和模型参数信息，save的对象是网络net
    if epoch - pre_epoch > max_interval_epoch:
        print('early stop')
        break

    #如果损失不载下降，则降低学习率
    if train_loss > pre_train_loss:
        lr = lr*lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    pre_train_loss = pre_train_loss
print("Done Training!")

| Wide-Resnet 4x10
Start Training...
the epoch 0, the train loss is 0.016856, the test loss is 0.014714, the test acc is 0.336573


/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Wide_ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type wide_basic. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


the epoch 1, the train loss is 0.015043, the test loss is 0.013887, the test acc is 0.361761
the epoch 2, the train loss is 0.014199, the test loss is 0.014546, the test acc is 0.370290
the epoch 3, the train loss is 0.013454, the test loss is 0.012009, the test acc is 0.474217
the epoch 4, the train loss is 0.012683, the test loss is 0.012831, the test acc is 0.481158
the epoch 5, the train loss is 0.012103, the test loss is 0.022918, the test acc is 0.432170
the epoch 6, the train loss is 0.011569, the test loss is 0.011221, the test acc is 0.564260
the epoch 7, the train loss is 0.011122, the test loss is 0.012112, the test acc is 0.546807
the epoch 8, the train loss is 0.010758, the test loss is 0.011963, the test acc is 0.560889
the epoch 9, the train loss is 0.010237, the test loss is 0.011549, the test acc is 0.576755
the epoch 10, the train loss is 0.010035, the test loss is 0.009231, the test acc is 0.639230
the epoch 11, the train loss is 0.009745, the test loss is 0.008701, 

the epoch 89, the train loss is 0.003853, the test loss is 0.004568, the test acc is 0.856010
the epoch 90, the train loss is 0.003764, the test loss is 0.003847, the test acc is 0.875843
the epoch 91, the train loss is 0.003739, the test loss is 0.004239, the test acc is 0.868306
the epoch 92, the train loss is 0.003720, the test loss is 0.005125, the test acc is 0.842523
the epoch 93, the train loss is 0.003731, the test loss is 0.004834, the test acc is 0.858191
the epoch 94, the train loss is 0.003734, the test loss is 0.004927, the test acc is 0.847679
the epoch 95, the train loss is 0.003665, the test loss is 0.005599, the test acc is 0.835383
the epoch 96, the train loss is 0.003617, the test loss is 0.004239, the test acc is 0.864935
the epoch 97, the train loss is 0.003624, the test loss is 0.005440, the test acc is 0.840341
the epoch 98, the train loss is 0.003665, the test loss is 0.003937, the test acc is 0.878620
the epoch 99, the train loss is 0.003532, the test loss is 0